# <font color=white><center><b>Next Word Prediction Using LSTM </center><br></font>
This notebook demonstrates a next-word prediction model trained on text data using an LSTM-based neural network. The steps include preprocessing the text, creating input-output sequences for training, building and training the model, and evaluating it using ROUGE scores.

## Import libraries

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from rouge import Rouge

## Read the text file

In [2]:
with open("The Modern Prometheus.txt", 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenizing the text

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1 # Total vocabulary size

## Creating input sequences for training

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

## Padding the sequences to uniform length

In [5]:
max_sequence_len = max(len(seq) for seq in input_sequences)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

## Splitting the data into features (X) and labels (y)

In [7]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Building and training the LSTM model

In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           760600    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 7606)              1148506   
                                                                 
Total params: 2,059,706
Trainable params: 2,059,706
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
2243/2243 [==============================] - 30s 11ms/step - loss: 6.4433 - accuracy: 0.0795
Epoch 2/50
2243/2243 [==============================] - 26s 12ms/step - loss: 5.7895 - accuracy: 0.1235
Epoch 3/50
2243/2243 [==============================] - 26s 12ms/step - loss: 5.4094 - accuracy: 0.1488
Epoch 4/50
2243/2243 [==============================] - 26s 12ms/step - loss: 5.0784 - accuracy: 0.1684
Epoch 5/50
2243/2243 [==============================] - 26s 12ms/step - loss: 4.7599 - accuracy: 0.1867
Epoch 6/50
2243/2243 [==============================] - 26s 11ms/step - loss: 4.4482 - accuracy: 0.2044
Epoch 7/50
2243/2243 [==============================] - 26s 12ms/step - loss: 4.1437 - accuracy: 0.2270
Epoch 8/50
2243/2243 [==============================] - 27s 12ms/step - loss: 3.8517 - accuracy: 0.2552
Epoch 9/50
2243/2243 [==============================] - 27s 12ms/step - loss: 3.5695 - accuracy: 0.2897
Epoch 10/50
2243/2243 [==============================] - 27s 12m

## Predicting the next n words for a given seed text

In [18]:
seed_text = "The master is"
next_words = 6

def predict_next_n_words(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

predict_next_n_words(seed_text, next_words)

1/1 [==============================] - 0s 33ms/step


'The master is a person of an excellent disposition'

## Generating test data from the text file

In [30]:
sentences = text.split('\n')

test_sentences = [s.strip() for s in sentences[-6:] if len(s.strip()) > 0]  # Select last 6 sentences for evaluation
reference_sentences = [" ".join(test_sentences[i].split()[:10]) for i in range(len(test_sentences))]

In [31]:
predictions = [predict_next_n_words(sentence, 5) for sentence in test_sentences]
predictions

1/1 [==============================] - 0s 23ms/step


['This website includes information about Project Gutenberg™, work b is the foundation',
 'including how to make donations to the Project Gutenberg Literary archive foundation “the name of',
 'Archive Foundation, how to help produce our new eBooks, and how to you to make the work',
 'subscribe to our email newsletter to hear about new eBooks. in amusement to be torn']

## Evaluating the model with ROUGE scores

In [33]:
rouge = Rouge()
scores = rouge.get_scores(predictions, reference_sentences, avg=True)

scores

{'rouge-1': {'r': 1.0, 'p': 0.6581959706959708, 'f': 0.7927414833293042},
 'rouge-2': {'r': 1.0, 'p': 0.6077922077922078, 'f': 0.755274931365785},
 'rouge-l': {'r': 1.0, 'p': 0.6581959706959708, 'f': 0.7927414833293042}}